In [11]:

#the '!' prefix to run shell commands (like pip install) in a Jupyter Notebook cell#
!pip install nba_api pandas

#import Libraries that are going to be used#
import pandas as pd #working with dataframes#
from nba_api.stats.static import players #used for specific players and to use their unique ID#
from nba_api.stats.endpoints import playergamelogs #for specific player game logs as only way I wont fetch pre-aggregated data like season averages#
import time #for code to work I cannot call API too much there is a limit#
import sys # imported for use in the !pip command, ensures installation#



Defaulting to user installation because normal site-packages is not writeable


In [46]:
#Configuration & Player ID Lookup#
PLAYER_NAME = "Jalen Brunson" #define players full name in#
# Define the last three seasons as strings for API call#
SEASONS = ['2023-24', '2022-23', '2021-22']
SEASONS_STR = ', '.join(SEASONS) #readable string for print statement ensuring confirming code works, not needed#

# get a list of all NBA players to find the required ID from API static data#
nba_players = players.get_players()

# Search the list for Jalen Brunson's ID#
#Use a list comprehension to filter the list of player dictionaries#
try:
    target_player = [
        player for player in nba_players 
        if player['full_name'] == PLAYER_NAME
    ][0]
    player_id = target_player['id'] #extract player ID#

#look for a dictionary where 'full_name' matches PLAYER_NAME#
    
    #print to ensure ID has been found#
    print(f"Player ID for {PLAYER_NAME} is: {player_id}")

#Error check incase name isnt found highly unlikely but just cautious could be a spelling error#
except IndexError:
    print(f"Error: Player '{PLAYER_NAME}' not found. Please check spelling or season.")
    player_id = None                         #stop to running code#


#Fetch Game Logs for Multiple Specificed Seasons#
#Start empty list to store DF for e/a Season#
all_logs = []

if player_id:
    #print statement to idicate what seasons for DF#
    print(f"Game logs for {SEASONS_STR}")
    

Player ID for Jalen Brunson is: 1628973
Game logs for 2023-24, 2022-23, 2021-22


In [53]:
#loop through e/a season defined in SEASONS defined at start#

for season in SEASONS:
        print(f"-> Fetching data for season: {season}...")
        try:

# actualize the endpoint class to get the data#
#Pass player ID#
#Pass current season sting in loop#            
            gamelogs = playergamelogs.PlayerGameLogs(
                player_id_nullable=player_id,
                season_nullable=season
            )
            
            # Pause for 1 second API rate limits prevents the script from making too many requests too quickly#
            time.sleep(1) 
            
            #Get the results as a DF#
            #API returns results in a dictionary; [0] selects the main data table#
            season_df = gamelogs.get_data_frames()[0]
            
            # Add the DataFrame to our list
            all_logs.append(season_df)

            #Confirmation Statement#
            print(f"   Successfully retrieved {len(season_df)} games for {season}.")
            
        except Exception as e:
            print(f"   An error occurred fetching data for {season}: {e}")
            
if all_logs:
    brunson_df = pd.concat(all_logs, ignore_index=True)
    #Concatenate all DF in the 'all_logs' list into a single DF#
    #'ignore_index=True' ensures a continuous index across all combined games#  
    #Display and Save Results#
    
    print("\n--- Combined Jalen Brunson Game Log Data (Head) ---")
    print(brunson_df.head())
    print(f"\nTotal Games Found Across All Seasons: {len(brunson_df)}")
        
    # Save the combined raw data locally for the file you would upload to GCS next
    file_name = f"{PLAYER_NAME.replace(' ', '_')}_multi_season_raw_data.csv"
    brunson_df.to_csv(file_name, index=False)
    print(f"Combined raw data saved locally: {file_name}")

else:
        #Brunson is rarely injsured for a whole season but other players might be, this accounts for that#
        print("No game logs were successfully retrieved.")
    

-> Fetching data for season: 2023-24...
   Successfully retrieved 94 games for 2023-24.
-> Fetching data for season: 2022-23...
   Successfully retrieved 83 games for 2022-23.
-> Fetching data for season: 2021-22...
   Successfully retrieved 100 games for 2021-22.

--- Combined Jalen Brunson Game Log Data (Head) ---
  SEASON_YEAR  PLAYER_ID    PLAYER_NAME NICKNAME     TEAM_ID  \
0     2021-22    1628973  Jalen Brunson    Jalen  1610612742   
1     2021-22    1628973  Jalen Brunson    Jalen  1610612742   
2     2021-22    1628973  Jalen Brunson    Jalen  1610612742   
3     2021-22    1628973  Jalen Brunson    Jalen  1610612742   
4     2021-22    1628973  Jalen Brunson    Jalen  1610612742   

  TEAM_ABBREVIATION         TEAM_NAME     GAME_ID            GAME_DATE  \
0               DAL  Dallas Mavericks  0042100315  2022-05-26T00:00:00   
1               DAL  Dallas Mavericks  0042100314  2022-05-24T00:00:00   
2               DAL  Dallas Mavericks  0042100313  2022-05-22T00:00:00   
3